In [22]:
import numpy as np
x = np.array([[2., 3., 1.]])
c = 0.1 * np.asarray([[0, 1]])
h = 0.1 * np.asarray([[2, 3]])

In [23]:
import tensorflow as tf

In [24]:
num_units = 2

args = np.concatenate((x,h), axis=1)
print(args)

[[2.  3.  1.  0.2 0.3]]


In [25]:
out_size = 4 * num_units
proj_size = args.shape[-1]
print(out_size)
print(proj_size)

8
5


In [26]:
weights = np.ones([proj_size, out_size]) * 0.5
print(weights)

[[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]]


In [27]:
out = np.matmul(args, weights)
print(out)

[[3.25 3.25 3.25 3.25 3.25 3.25 3.25 3.25]]


In [28]:
bias = np.ones([out_size]) * 0.5
print(bias)

[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]


In [29]:
concat = out + bias
print(concat)

[[3.75 3.75 3.75 3.75 3.75 3.75 3.75 3.75]]


In [30]:
i, j, f, o = np.split(concat, 4, 1)
print(i)
print(j)
print(f)
print(o)

[[3.75 3.75]]
[[3.75 3.75]]
[[3.75 3.75]]
[[3.75 3.75]]


In [31]:
g = np.tanh(j)
print(g)

[[0.99889444 0.99889444]]


In [32]:
def sigmoid_array(x):
    return 1 / (1 + np.exp(-x))

In [33]:
forget_bias = 1.0

sigmoid_f = sigmoid_array(f + forget_bias)
print(sigmoid_f)

[[0.99142251 0.99142251]]


In [34]:
sigmoid_array(i) * g

new_c = c * sigmoid_f + sigmoid_array(i) * g
print(new_c)

[[0.97594248 1.07508473]]


In [35]:
new_h = np.tanh(new_c) * sigmoid_array(o)
print(new_h)

[[0.73404107 0.77318564]]


In [36]:
print(new_h)
print(new_c)

[[0.73404107 0.77318564]]
[[0.97594248 1.07508473]]


In [21]:
inputs = tf.placeholder(shape=(max_time, batch_size, input_depth),
                        dtype=tf.float32)
sequence_length = tf.placeholder(shape=(batch_size,), dtype=tf.int32)
inputs_ta = tf.TensorArray(dtype=tf.float32, size=max_time)
inputs_ta = inputs_ta.unstack(inputs)

cell = tf.contrib.rnn.LSTMCell(num_units)

def loop_fn(time, cell_output, cell_state, loop_state):
  emit_output = cell_output  # == None for time == 0
  if cell_output is None:  # time == 0
    next_cell_state = cell.zero_state(batch_size, tf.float32)
  else:
    next_cell_state = cell_state
  elements_finished = (time >= sequence_length)
  finished = tf.reduce_all(elements_finished)
  next_input = tf.cond(
      finished,
      lambda: tf.zeros([batch_size, input_depth], dtype=tf.float32),
      lambda: inputs_ta.read(time))
  next_loop_state = None
  return (elements_finished, next_input, next_cell_state,
          emit_output, next_loop_state)

outputs_ta, final_state, _ = raw_rnn(cell, loop_fn)
outputs = outputs_ta.stack()

NameError: name 'max_time' is not defined

In [ ]:
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)